In [1]:
links = ["A: B C D", "B: A D", "C: A", "D: B C"]
deadEnds = ["A: B C D", "B: A D", "C:", "D: B C"]
spiderTraps = ["A: B C D", "B: A D", "C: C", "D: B C"]

rawTextRDD = sc.parallelize(deadEnds) # ["A: B C D", "B: A D", "C:", "D: B C"]
initialRank = 1.0 / rawTextRDD.count() # 0.25 = 1/4


In [2]:
def initialMapper(raw):
  rawArray = raw.split(":")
  page = rawArray[0]
  links = rawArray[1].strip().split(" ")
  return (page, (links, initialRank))

def linksMapper(raw):
  rawArray = raw.split(":")
  page = rawArray[0]
  links = rawArray[1].strip().split(" ")
  return (page, links)

In [3]:
linksRDD = rawTextRDD.map(linksMapper)
#linksRDD.cache()
linksRDD.take(5)

Out[ 21 ]: [('A', ['B', 'C', 'D']), ('B', ['A', 'D']), ('C', ['']), ('D', ['B', 'C'])]

In [4]:
pageRankRDD = rawTextRDD.map(initialMapper)
pageRankRDD.take(5)

Out[ 22 ]: [('A', (['B', 'C', 'D'], 0.25)),
 ('B', (['A', 'D'], 0.25)),
 ('C', ([''], 0.25)),
 ('D', (['B', 'C'], 0.25))]

In [5]:
def contributionMapper(row):
  page = row
  links = row[1][0]
  contribution = row[1][1] / len(links)
  pagesAddedValue = []
  for link in links:
    pagesAddedValue.append( (link, contribution) )
  return pagesAddedValue


In [6]:
individualContributionRDD = pageRankRDD.flatMap(contributionMapper)
individualContributionRDD.take(10)

Out[ 25 ]: [('B', 0.08333333333333333),
 ('C', 0.08333333333333333),
 ('D', 0.08333333333333333),
 ('A', 0.125),
 ('D', 0.125),
 ('', 0.25),
 ('B', 0.125),
 ('C', 0.125)]

In [7]:
ranksRDD = individualContributionRDD.reduceByKey(lambda a, b: a+b)
ranksRDD.take(5)

Out[ 28 ]: [('A', 0.125),
 ('', 0.25),
 ('C', 0.20833333333333331),
 ('B', 0.20833333333333331),
 ('D', 0.20833333333333331)]

In [8]:
pageRankRDD = linksRDD.join(ranksRDD)
pageRankRDD.take(5)

Out[ 29 ]: [('A', (['B', 'C', 'D'], 0.125)),
 ('C', ([''], 0.20833333333333331)),
 ('B', (['A', 'D'], 0.20833333333333331)),
 ('D', (['B', 'C'], 0.20833333333333331))]

In [9]:
iter = 15
for i in range(1, iter):
  ranksRDD = pageRankRDD.flatMap(contributionMapper).reduceByKey(lambda a, b: a+b)
  pageRankRDD = linksRDD.join(ranksRDD)
#pageRankRDD.take(5)

In [10]:
sortedPageRankRDD = pageRankRDD.sortBy(lambda pair: -pair[1][1])
sortedPageRankRDD.collect()

Out[ 30 ]: [('C', ([''], 0.20833333333333331)),
 ('B', (['A', 'D'], 0.20833333333333331)),
 ('D', (['B', 'C'], 0.20833333333333331)),
 ('A', (['B', 'C', 'D'], 0.125))]

In [11]:
links = ["A: B C D", "B: A D", "C: A", "D: B C"]
deadEnds = ["A: B C D", "B: A D", "C:", "D: B C"]
spiderTraps = ["A: B C D", "B: A D", "C: C", "D: B C"]

rawTextRDD = sc.parallelize(deadEnds)  # ["A: B C D", "B: A D", "C:", "D: B C"]
initialRank = 1.0 / rawTextRDD.count()  # 0.25 = 1/4


########################### IMPLEMENTATIONS ##################################

def init_maper(raw):  # A: B C D
    rawArray = raw.split(":")  # A , B C D
    page = rawArray[0]  # A
    links = rawArray[1].strip().split(" ")  # B C D
    return page, (links, initialRank)


def enlaces_mapper(raw):  # A: B C D
    rawArray = raw.split(":")  # A , B C D
    page = rawArray[0]  # A
    links = rawArray[1].strip().split(" ")  # B C D
    return page, links


links_rdd = rawTextRDD.map(enlaces_mapper)
page_rank_RDD = rawTextRDD.map(init_maper)

def contr_mapper(row):
    links = row[1][0]  # B C D
    contribution = row[1][1] / len(links)  # number
    pagesAddedValue = []
    for link in links:
        pagesAddedValue.append((link, contribution))  # filling pagesAddedValue
    return pagesAddedValue


contributions_RDD = page_rank_RDD.flatMap(contr_mapper)

def function_suma(a, b):
    return a + b


rank_RDD = contributions_RDD.reduceByKey(function_suma)
page_rank_RDD = links_rdd.join(rank_RDD)

iter = 15
for i in range(1, iter):
    rank_RDD = page_rank_RDD.flatMap(contr_mapper).reduceByKey(function_suma)
    page_rank_RDD = links_rdd.join(rank_RDD)

sortedPageRankRDD = page_rank_RDD.sortBy(lambda pair: -pair[1][1])
sortedPageRankRDD.collect()

######################## THE SOLUTION HAS TO BE: ##############################
# [('C', ([''], 0.00240619428600049)),
#  ('B', (['A', 'D'], 0.00240619428600049)),
#  ('D', (['B', 'C'], 0.00240619428600049)),
#  ('A', (['B', 'C', 'D'], 0.001650987720184256))]


Out[ 34 ]: [('C', ([''], 0.00240619428600049)),
 ('B', (['A', 'D'], 0.00240619428600049)),
 ('D', (['B', 'C'], 0.00240619428600049)),
 ('A', (['B', 'C', 'D'], 0.001650987720184256))]